## Create table in BigQuery

In [38]:
%%bigquery --use_rest_api
CREATE OR REPLACE TABLE lics.prices_azure (
    currencyCode STRING, 
    retailPrice FLOAT64, 
    unitPrice FLOAT64, 
    armRegionName STRING,
    meterId STRING,
    meterName STRING,
    skuName STRING,
    productName STRING,
    serviceFamily STRING,
    unitOfMeasure STRING,
    type STRING,
    armSkuName STRING,
    licensePrice FLOAT64
);    

""


## Create view

In [42]:
%%bigquery --use_rest_api
CREATE OR REPLACE VIEW lics.v_prices_azure
AS 
    SELECT armRegionName, meterName, productName, licensePrice, licensePrice / Cores AS licensePriceCore
    FROM (
        SELECT *,
            CASE
                WHEN armSkuName = 'Basic_A0' THEN 1
                WHEN armSkuName = 'Basic_A1' THEN 1
                WHEN armSkuName = 'Basic_A2' THEN 2
                WHEN armSkuName = 'Basic_A3' THEN 4
                WHEN armSkuName = 'Basic_A4' THEN 8
                WHEN armSkuName = 'Standard_A0' THEN 1
                WHEN armSkuName = 'Standard_A1' THEN 1
                WHEN armSkuName = 'Standard_A2' THEN 2
                WHEN armSkuName = 'Standard_A3' THEN 4
                WHEN armSkuName = 'Standard_A4' THEN 8
                WHEN armSkuName = 'Standard_A5' THEN 2
                WHEN armSkuName = 'Standard_A6' THEN 4
                WHEN armSkuName = 'Standard_A7' THEN 8
                WHEN armSkuName = 'Standard_A8' THEN 8
                WHEN armSkuName = 'Standard_A9' THEN 16
                WHEN armSkuName = 'Standard_A10' THEN 8
                WHEN armSkuName = 'Standard_A11' THEN 16
                WHEN armSkuName = 'Standard_D1' THEN 1
                WHEN armSkuName = 'Standard_D2' THEN 2
                WHEN armSkuName = 'Standard_D3' THEN 4
                WHEN armSkuName = 'Standard_D4' THEN 8
                WHEN armSkuName = 'Standard_D11' THEN 2
                WHEN armSkuName = 'Standard_D12' THEN 4
                WHEN armSkuName = 'Standard_D13' THEN 8
                WHEN armSkuName = 'Standard_D14' THEN 16
                WHEN armSkuName = 'Standard_DS1' THEN 1
                WHEN armSkuName = 'Standard_DS2' THEN 2
                WHEN armSkuName = 'Standard_DS3' THEN 4
                WHEN armSkuName = 'Standard_DS4' THEN 8
                WHEN armSkuName = 'Standard_DS11' THEN 2
                WHEN armSkuName = 'Standard_DS12' THEN 4
                WHEN armSkuName = 'Standard_DS13' THEN 8
                WHEN armSkuName = 'Standard_DS14' THEN 16
                WHEN armSkuName = 'Standard_GS1' THEN 2
                WHEN armSkuName = 'Standard_GS2' THEN 4
                WHEN armSkuName = 'Standard_GS3' THEN 8
                WHEN armSkuName = 'Standard_GS4' THEN 16
                WHEN armSkuName = 'Standard_GS5' THEN 32
                WHEN armSkuName = 'Standard_G1' THEN 2
                WHEN armSkuName = 'Standard_G2' THEN 4
                WHEN armSkuName = 'Standard_G3' THEN 8
                WHEN armSkuName = 'Standard_G4' THEN 16
                WHEN armSkuName = 'Standard_G5' THEN 32
                ELSE CAST(REGEXP_EXTRACT(meterName, r'^[a-zA-Z]+([0-9]+)') AS INT64)
            END AS Cores
        FROM lics.prices_azure
    )
;

""


## Query Azure Retail Prices API
More information in the [](https://docs.microsoft.com/en-us/rest/api/cost-management/retail-prices/azure-retail-prices).

In [39]:
import requests
import json
from google.cloud import bigquery

def getPrices(region, filterExpression = None):
    uri = f"https://prices.azure.com/api/retail/prices?$filter=serviceName eq 'Virtual Machines'"
    uri += f" and armRegionName eq '{region}'"
    uri += f" and indexof(meterName, 'Spot') eq -1"
    uri += f" and indexof(meterName, 'Low Priority') eq -1"
    uri += f" and indexof(armSkuName, 'Promo') eq -1"
    
    if filterExpression is not None:
        uri += f" {filterExpression}"
    
    return invokePriceApi(uri)
    
def invokePriceApi(uri):
    result = requests.get(uri)
    
    if result.status_code == 200:
        data = result.json()
        items = data["Items"]
        count = data["Count"]
        nextPageLink = data["NextPageLink"]
        
        while nextPageLink is not None:
            result = requests.get(nextPageLink)
            data = result.json()
            items.extend(data["Items"])
            nextPageLink = data["NextPageLink"]
        
        return items

def getWindowsSkus(region):
    return getPrices(region, "and contains(ProductName, 'Windows')")
    
def getConsumptionPrice(items):
    for item in items:
        if item["type"] == "Consumption":
            return item["unitPrice"]
        
    return None

# Initiate BigQuery
bq = bigquery.Client()
region = "westeurope"

windowsSkus = getWindowsSkus(region)

if len(windowsSkus) > 0:
    query = "INSERT INTO lics.prices_azure (currencyCode, retailPrice, unitPrice, armRegionName, meterId, meterName, skuName, productName, serviceFamily, unitOfMeasure, type, armSkuName, licensePrice) VALUES "
    values = ""

    for item in windowsSkus:
        meterName = item["meterName"]
        
        if item["type"] == 'Consumption':
            skuId = item["skuId"]
            currencyCode = item["currencyCode"]
            retailPrice = item["retailPrice"]
            unitPrice = item["unitPrice"]
            armRegionName = item["armRegionName"]
            meterId = item["meterId"]
            skuName = item["skuName"]
            productName = item["productName"]
            serviceFamily = item["serviceFamily"]
            unitOfMeasure = item["unitOfMeasure"]
            meterType = item["type"]
            armSkuName = item["armSkuName"]
            
            rawSkus = getPrices(region, f"and meterName eq '{meterName}' and indexof(productName, 'Windows') eq -1")
            rawPrice = getConsumptionPrice(rawSkus)
            
            if rawPrice is None:
                print(f"Unit price without Windows license could not be retrieved ({productName} - {meterName})")
                continue
            
            licensePrice = unitPrice - rawPrice
            
            print(f"{meterName} - {armSkuName}: {licensePrice} / excl. lics: {rawPrice} / incl. lics: {unitPrice}")
            
            if len(values) > 0:
                values += ", "
            
            values += f"('{currencyCode}', {retailPrice}, {unitPrice}, '{armRegionName}', '{meterId}', '{meterName}', '{skuName}', '{productName}', '{serviceFamily}', '{unitOfMeasure}', '{meterType}', '{armSkuName}', {licensePrice})"
            
    query += values
    bq.query(query).result()
else:
    print(f"Empty list for {region}")

F2s - Standard_F2s: 0.09199999999999998 / excl. lics: 0.114 / incl. lics: 0.206
F2/F2s - Standard_F2: 0.09199999999999998 / excl. lics: 0.114 / incl. lics: 0.206
E48 v3/E48s v3 - Standard_E48_v3: 2.2080000000000006 / excl. lics: 3.647 / incl. lics: 5.855
E48s v3 - Standard_E48s_v3: 2.2080000000000006 / excl. lics: 3.647 / incl. lics: 5.855
F4/F4s - Standard_F4: 0.18399999999999997 / excl. lics: 0.227 / incl. lics: 0.411
F4s - Standard_F4s: 0.18399999999999997 / excl. lics: 0.227 / incl. lics: 0.411
L32s - Standard_L32s: 1.4720000000000004 / excl. lics: 2.976 / incl. lics: 4.448
NC6 - Standard_NC6: 0.22100000000000009 / excl. lics: 1.166 / incl. lics: 1.387
D32a v4/D32as v4 - Standard_D32a_v4: 1.4719999999999998 / excl. lics: 1.84 / incl. lics: 3.312
D32as v4 - Standard_D32as_v4: 1.4719999999999998 / excl. lics: 1.84 / incl. lics: 3.312
G3/GS3 - Standard_G3: 0.28000000000000025 / excl. lics: 2.8 / incl. lics: 3.08
GS3 - Standard_GS3: 0.28000000000000025 / excl. lics: 2.8 / incl. lics: 3